In [0]:
import keras
from keras.layers import Flatten,Conv2D,Dense,MaxPooling2D,InputLayer,Activation,GlobalAveragePooling2D, Input, Concatenate, Flatten, Add
from keras import Model, Sequential

def residual_bottleneck(x,n_reduction,n_expansion,name):
  x_mod=x
  x_mod = Conv2D(n_reduction,(1,1),padding="same",activation="relu",name=f"{name}_reduce")(x_mod)
  x_mod = Conv2D(n_reduction,(3,3),padding="same",activation="relu",name=f"{name}_conv")(x_mod)
  x_mod = Conv2D(n_expansion,(1,1),padding="same",activation=None,name=f"{name}_expand")(x_mod)
  new_x = Add(name=f"{name}_add")([x, x_mod])
  new_x = Activation("relu",name=f"{name}_relu")(new_x) # mismas dimensiones
  return new_x


def ResNet50(input_shape,classes):
  input = Input(shape=input_shape,name="input")
  x = Conv2D(64,(7,7),strides=(2,2),padding="same",activation="relu",name="c1")(input)
  reductions = [64,128,256,512]
  blocks     = [3,4,6,3]
  for i,(block_n,reduction) in enumerate(zip(blocks,reductions)):
    expansion=reduction*2
    #Distinto al original; simplificado
    x = Conv2D(expansion,(3,3),strides=(2,2),padding="same",activation="relu",name=f"block{i}_downsample")(x)
    for j in range(block_n):
      x = residual_bottleneck(x,reduction,expansion,f"block{i}_{j}")
  x = GlobalAveragePooling2D(name="gap")(x)
  x = Dense(classes,activation="softmax",name="fc1")(x)
  model = Model(inputs=[input],outputs=[x])
  return model

input_shape=(224,224,3)
classes=10
model=ResNet50(input_shape,classes)  
print(model.summary())


Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
c1 (Conv2D)                     (None, 112, 112, 64) 9472        input[0][0]                      
__________________________________________________________________________________________________
block0_downsample (Conv2D)      (None, 56, 56, 128)  73856       c1[0][0]                         
__________________________________________________________________________________________________
block0_0_reduce (Conv2D)        (None, 56, 56, 64)   8256        block0_downsample[0][0]          
___________________________________________________________________________________________